In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
library(forcats)

### 0 - Read exhaustive analysis

In [ ]:
fe <- fread(paste0(SHARE_DIR, "0_run_fe.csv"))

In [ ]:
remove_last <- function(i) i[-length(i)]

In [ ]:
base <- 
fe %>% 
 rw() %>% 
 mu( raw = paste(remove_last(strsplit(feature, "_")[[1]]), collapse = "_")) %>% 
 ug() %>% 
 gb(cohortGo, raw, fisher_pval) %>% 
 mu( rk = row_number(desc(e_nr)), 
     label = 
     factor(ifelse(pr_nr_given_e > pr_nr_overall, 
                   "Non-Response Associated", "Response Associated"), 
     levels = c("Response Associated", "Non-Response Associated"))) %>%
 fi(rk == 1) %>% se(-rk) %>% 
 gb( cohortGo, label ) %>% 
 fi( !grepl("geneset_mp", feature), 
     cohortGo != "Pan-Cancer ## Folinic acid ## Platinum ## Pyrimidine (ant)agonist ## Topoisomerase inhibitor") %>% 
 se( cohortGo, label, feature, fisher_pval, e_nr, e_r, ne_nr, ne_r) %>% 
 fi(grepl("rna_", feature)) %>% 
  mu(
    rna = ifelse(m("rna_", feature), "RNA Feature", "DNA or Clinical Feature"),
    feature = ifelse(m("rna_geneset", feature), strsplit(feature, "rna_geneset_")[[1]][2], feature), 
    feature = ifelse(m("gene_set_", feature), strsplit(feature, "gene_set_")[[1]][2], feature),
    feature = str_to_title(feature)) 

- Remove redundancies in Fisher exact output
- Some tests are equivalent but signs are flipped

In [ ]:
map <- 
c("e_nr" = "Event + No DCB", 
  "e_r" = "Event + DCB", 
  "ne_nr" = "No Event + No DCB", 
  "ne_r" = "No Event + DCB")
remove_last <- function(i) i[-length(i)]

In [ ]:
ready <- 
base %>% 
 ga( event, ct, -cohortGo, -label, -feature, -fisher_pval ) %>% 
 rw() %>% 
 mu(event2 = factor(map[[event]], levels = unname(map))) 

### Filters 

In [ ]:
base %>% 
 gb(label, event2, rna, sv = grepl("svtmb", feature)) %>% 
 mu(rk = row_number(fisher_pval)) %>% 
 fi(rk < 100) %>% 
 mu(event = !m("No Event", event2), dcb = !m("No DCB", event2)) %>% 
 ug()

In [ ]:
summary(base$rk)

In [ ]:
index_no_dcb <- base %>% fi(event2 == "Event + DCB", ct == 0) %>% se(cohortGo, label, feature)
ready <- base %>% ij(index_no_dcb, by = c("cohortGo", "label", "feature"))

In [ ]:
ready

- Prepare drivers

In [ ]:
highlight_drivers <- 
c("Driver_pathway_immune_evasion_gt50" = "Driver Immune Evasion (8 B2M events)", 
  "Driver_pathway_hippo_gt50" = "Driver Hippo Pathway (6 NF2 events)", 
  "Driver_erbb4" = "Driver HER4", 
  "Driver_fbxw7" = "Driver FBXW7")

In [ ]:
ready_driver <- 
ready %>% 
  fi(feature %in% names(highlight_drivers)) %>%
  rw() %>% 
  mu(feature = highlight_drivers[[feature]]) %>% 
  ug()

### Figure Preparation

In [ ]:
alphas <- c("FALSE" = .4, "TRUE" = 1)
colors <- c("FALSE" = "grey", "TRUE" = "black")
response <- c("FALSE" = "#e52f28", "TRUE" = "#7AABD3")

In [ ]:
labels = ready %>% gb(cohortGo, label = formatC(fisher_pval, format = "e", digits = 1)) %>% su(y_pos = max(ct)*1.05)

In [ ]:
most <- max(ready$ct) + 1.05 

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 10)

In [ ]:
top <- 
ready %>% 
 ggplot(aes(x = event2, y = ct, alpha = event, fill = dcb, color = event)) + 
 facet_wrap(~fct_reorder(cohortGo, fisher_pval, .fun = mean) + feature , ncol = 4, scales = "free") +    
  geom_bar(stat = "identity") + 
  scale_fill_manual( values = response) +  
  scale_alpha_manual( values = alphas) + 
  scale_color_manual( values = colors) + 
  go_theme + 
  geom_text(aes(label = ct), vjust = -0.5, color = "black", size = 3) + 
  geom_text(aes(x = 1.6, 
               y = 70, 
               label = paste0("p-value: ", formatC(fisher_pval, format = "e", digits = 1))), 
               vjust = -0.5, color = "black", size = 3) + 
  coord_cartesian(ylim = c(0,80)) + 
  labs(title = "Highlighted Driver Events with No Durable Benefit given Treatment", y = "Number of Patients\n", x = "Event and Durable Clinical Benefit") + 
  theme(strip.background = element_rect(fill = "white", color = "black"), legend.position = "none")

In [ ]:
options(repr.plot.width = 11, repr.plot.height = 3.5)

In [ ]:
top

In [ ]:
ggsave( paste0(FIG_DIR, "Driver_events_with_no_durable_benefit.png"), plot = top, width = 11, height = 3.5)

In [ ]:
paste0(FIG_DIR, "Driver_events_with_no_durable_benefit.png")